<H1> Dashboard

A simple dashboard created using Plotly Dash to showcase the genre distribution, genre sales, platform sales and sales over time for all publishers together, Nintendo, Electronic Arts and Activision who are the top three publishers sales-wise.

Because the dashboard is unable to work offline, I have included a demo of it [here](https://drive.google.com/file/d/1gF93zEcDrPQ1qaBRzofjeAZZjK4BsZJp/view?usp=share_link), with the syntax below.

In [2]:
pip install jupyter-dash

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade plotly

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install plotly-express

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install dash --upgrade

Note: you may need to restart the kernel to use updated packages.


In [6]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [7]:
vgsales_df = pd.read_csv('/Users/jojotan/Documents/Portfolio/Video_Games/clean_vgsales.csv')
vgsales_df.head()

,Rank,Name,Platform,Year,Genre,Main Publisher,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,Nintendo,11.27,8.89,10.22,1.00,31.37


In [8]:
genre_count = vgsales_df['Genre'].value_counts()
genre_count

genre_count = pd.pivot_table(vgsales_df,
                             index = ['Genre','Main Publisher'],
                             values=["Name"],
                             aggfunc=[len])

genre_count = genre_count.reset_index()
genre_count.columns = ['Genre', 'Main Publisher', 'Count']

top_3 = ['Nintendo', 'Electronic Arts', 'Activision']
genre_count2 = genre_count[genre_count['Main Publisher'].isin(top_3)]

In [9]:
top3_df = vgsales_df[vgsales_df['Main Publisher'].isin(top_3)]

In [10]:
yearly_sales = pd.pivot_table(vgsales_df,
                              index = ['Year'],
                              values = ['Global_Sales'],
                              aggfunc = [np.sum])

yearly_sales = yearly_sales.reset_index()
yearly_sales.columns = ['Year', 'Total Sales']

In [11]:
top3_yearly_sales = vgsales_df[vgsales_df['Main Publisher'].isin(top_3)]

top3_yearly_sales = pd.pivot_table(top3_yearly_sales,
                                   index = ['Main Publisher', 'Year'],
                                   values = ['Global_Sales'],
                                   aggfunc = [np.sum])

top3_yearly_sales = top3_yearly_sales.reset_index()
top3_yearly_sales.columns = ['Main Publisher', 'Year', 'Total Sales']
top3_yearly_sales.head()

,Main Publisher,Year,Total Sales
0,Activision,1980,3.85
1,Activision,1981,8.50
2,Activision,1982,1.86
3,Activision,1983,1.94
4,Activision,1984,0.27


In [12]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(   
    html.Div([
    html.H1('Video Game Publishers Records',style = {'textAlign': 'center', 'color': '#000000', 'font-size': 40}),
    dcc.Dropdown(id = 'publishers',
        options = ['All Publishers', 'Nintendo', 'Electronic Arts', 'Activision'],
        value = 'All Publishers',
        searchable = True),
        html.Br(),    
    
    html.H4('Genre Distribution'),
    dcc.Graph(id = 'genre-dist-pie-chart', 
              style = {'textAlign': 'center', 
                       'margin': 'auto',
                       'width': 1000,
                       'height': 700}),
        html.Br(),
    
    html.H4('Genre Sales'),
    dcc.Graph(id = 'genre-sales-bar-chart',
              style = {'textAlign': 'center', 
                       'margin': 'auto',
                       'width': 1000,
                       'height': 700}),
        html.Br(),
    
    html.H4('Platform Sales'),
    dcc.Graph(id = 'platform-sales-bar-chart',
              style = {'textAlign': 'center', 
                       'margin': 'auto',
                       'width': 1000,
                       'height': 700}),
        html.Br(),
    
    html.H4('Sales Over Time'),
    dcc.Graph(id = 'sales-over-time-line-chart',
              style = {'textAlign': 'center', 
                       'margin': 'auto',
                       'width': 1000,
                       'height': 700}),
        html.Br(),
    ], style={'textAlign': 'center'})
    )

@app.callback(
        Output(component_id = 'genre-dist-pie-chart', component_property = 'figure'),
        Input(component_id = 'publishers', component_property = 'value'))

def get_pie_chart(publishers):
    if publishers == 'All Publishers':
        fig = px.pie(genre_count,
                     values = 'Count',
                     names = 'Genre',
                     title = 'All Publishers')
        return fig

    else:
        df = genre_count2[genre_count2['Main Publisher'] == publishers]
        fig = px.pie(df,
                     values = 'Count',
                     names = 'Genre',
                     title = f'{publishers}')
        return fig

@app.callback(
        Output(component_id = 'genre-sales-bar-chart', component_property = 'figure'),
        Input(component_id = 'publishers', component_property = 'value'))

def get_bar_chart(publishers):
    if publishers == 'All Publishers':
        fig = px.histogram(vgsales_df,
                           x = 'Genre',
                           y = 'Global_Sales',
                           title = 'All Publishers')
        fig.update_traces(marker_color='#dbb40c')
        return fig
    
    else:
        mask = top3_df['Main Publisher'] == publishers
        fig = px.histogram(top3_df[mask], 
                           x = 'Genre', 
                           y = 'Global_Sales',
                           title = f'{publishers}')
        fig.update_traces(marker_color='#dbb40c')
        return fig

@app.callback(
        Output(component_id = 'platform-sales-bar-chart', component_property = 'figure'),
        Input(component_id = 'publishers', component_property = 'value'))

def get_bar_chart(publishers):
    if publishers == 'All Publishers':
        fig = px.histogram(vgsales_df,
                           x = 'Global_Sales',
                           y = 'Platform',
                           title = 'All Publishers',
                           orientation = 'h')
        fig.update_traces(marker_color='#cb416b')
        return fig
    
    else:
        mask = top3_df['Main Publisher'] == publishers
        fig = px.histogram(top3_df[mask], 
                           x = 'Global_Sales', 
                           y = 'Platform',
                           title = f'{publishers}',
                           orientation = 'h')
        fig.update_traces(marker_color='#cb416b')
        return fig

@app.callback(
        Output(component_id = 'sales-over-time-line-chart', component_property = 'figure'),
        Input(component_id = 'publishers', component_property = 'value'))

def get_line_chart(publishers):
    if publishers == 'All Publishers':
        fig = px.line(yearly_sales,
                      x = 'Year',
                      y = 'Total Sales',
                      title = 'All Publishers')
        fig.update_traces(line_color='#8eab12')
        return fig
    
    else:
        mask = top3_yearly_sales['Main Publisher'] == publishers
        fig = px.line(top3_yearly_sales[mask], 
                      x = 'Year', 
                      y = 'Total Sales',
                      title = f'{publishers}')
        fig.update_traces(line_color='#8eab12')
        return fig



if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
